In [5]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, \
    Flatten, Lambda, LSTM, RepeatVector, TimeDistributed, Reshape, \
    Conv2D, MaxPooling2D, BatchNormalization, ConvLSTM2D, Bidirectional, Masking
import keras.callbacks as Callbacks 
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import scale
from matplotlib import pyplot as plt

In [6]:
from skimage.util import random_noise
from skimage.transform import rotate
from scipy import ndimage

In [7]:
data = h5py.File("./data_sample.hdf5", "r")
print("data keys: " + str(list(data.keys())))

data keys: ['images', 'labels', 'observation_days']


In [8]:
images = data["images"][:]
print("image shape: (sample, x_size, y_size, epoch) = " + str(images.shape))
labels = data["labels"][:]

image shape: (sample, x_size, y_size, epoch) = (72000, 21, 21, 48)


In [9]:
def preprocess_img(images, percent=0.25):
    
    size, timestep, _, _ = images.shape
    sample_size = int(size * percent)
    
    np.random.seed(209)
    # pick the sequence with replacement
    sample_seq_indices = np.random.choice(np.arange(sample_size), sample_size)
    # pick the timestep
    sample_inseq_indices = np.random.choice(np.arange(timestep), sample_size)
    # pick the aug mean
    sample_aug_type = np.random.choice(np.arange(4), sample_size)
    
    for idx in range(sample_size):
        i, j, k = sample_seq_indices[idx], \
            sample_inseq_indices[idx], sample_aug_type[idx]
        if k == 0: # rotate
            images[i, j] = rotate(images[i, j], 45)
        elif k == 1: # random noise
            images[i, j] = random_noise(images[i, j], seed=209)
        elif k == 2: # horizontal flip
            images[i, j] = images[i, j][:,::-1]
        else: # vertical flip
            images[i, j] = images[i, j][::-1,:]

In [10]:
def scale_images(images):
    t_images = np.transpose(images, (0,3,1,2))
    rt_images = t_images.reshape(72000*48, 21, 21)
    max_per_img = np.max(rt_images.reshape(-1, 21*21), axis=1, keepdims=1)
    scaled_images = rt_images.reshape(-1, 21*21) / max_per_img
    scaled_images = scaled_images.reshape(-1, 21, 21).reshape(-1, 48, 21, 21)
    return scaled_images

In [11]:
def txt2digit(labels):
    dic = {'Asteroids':0, 'Constant':1, 'EmptyLigh':2, 'M33Cephei':3, 'RRLyrae':4, 'Supernova':5}
    labels_digit = np.array([dic[i] for i in labels])
    return labels_digit

In [12]:
def build_dataset(images, labels):
    scaled_img = scale_images(images)
    preprocess_img(scaled_img)
    x = np.expand_dims(scaled_img, len(scaled_img.shape))
    y = to_categorical(txt2digit(labels))
    return x, y

In [13]:
x, y = build_dataset(images, labels)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=209, stratify=y)

In [14]:
def init_model(cnn_input_dim=21, cnn_output_dim=128, cnn_dropout=0.5,
               rnn_hidden_dim=128, rnn_output_dim=64, num_classes=6, rnn_dropout=0.5, timestep=48):
    
    
    intput_shape = (timestep, cnn_input_dim, cnn_input_dim, 1)
    model = Sequential()
    # CNN
    model.add(TimeDistributed(Conv2D(32, (4,4), \
                     padding='same', activation='relu', kernel_initializer='uniform'), \
                              input_shape=intput_shape))
    
    model.add(BatchNormalization())
    # model.add(Dropout(cnn_dropout))
    
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    
    #model.add(BatchNormalization())
    
    model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    
    #model.add(BatchNormalization())
    
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    
    #model.add(BatchNormalization())
    
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    
    #model.add(BatchNormalization())
#     model.add(Dropout(cnn_dropout))
    
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    #model.add(Dropout(cnn_dropout))
    model.add(BatchNormalization())
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Flatten()))
    model.add(TimeDistributed(Dense(128, activation='relu')))
    
    #model.add(BatchNormalization())
#     model.add(Dropout(cnn_dropout))
    
    model.add(TimeDistributed(Dense(cnn_output_dim, activation='relu')))
    
    model.add(Bidirectional(LSTM(rnn_hidden_dim, dropout=rnn_dropout, return_sequences=True), \
                          input_shape=(timestep, cnn_output_dim)))
    # repeat vector for timestep
    #model.add(RepeatVector(timestep))
    # decode
    model.add(Bidirectional(LSTM(rnn_hidden_dim, dropout=rnn_dropout)))

    model.add(Dense(num_classes, activation='softmax'))
    
    #out = rnn(cat)
    
    return model#Model(inputs=iL, outputs=out)

In [15]:
def init_model3(cnn_input_dim=21, cnn_output_dim=128, cnn_dropout=0.5,
               rnn_hidden_dim=128, rnn_output_dim=64, num_classes=6, rnn_dropout=0.5, timestep=48,
               bidir_mode='concat'):
    
    input_shape = (timestep, cnn_input_dim, cnn_input_dim, 1, )
    
    model = Sequential()
    
    model.add(TimeDistributed(Masking(), input_shape=input_shape, name='Masking'))
    model.add(TimeDistributed(Conv2D(48, (4,4), \
                     padding='same', activation='relu', kernel_initializer='uniform'), \
                              input_shape=input_shape, name='Conv2D_1'))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1)), name='MaxPooling2D_1'))
    
    model.add(TimeDistributed(Conv2D(24, (3,3), padding='same', activation='relu', kernel_initializer='uniform'),\
                 name='Conv2D_2'))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1)), name='MaxPooling2D_2'))

    model.add(TimeDistributed(Conv2D(12, (3,3), padding='same', activation='relu', kernel_initializer='uniform'), \
                             name='Conv2D_3'))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1)), name='MaxPooling2D_3'))
    
    model.add(TimeDistributed(Flatten(), name='Faltten'))

    model.add(TimeDistributed(Dense(128, activation='relu'), name='Dense_128'))
    model.add(Dropout(0.25))
    
    model.add(Bidirectional(LSTM(rnn_hidden_dim, dropout=rnn_dropout, return_sequences=True), \
                          input_shape=(timestep, cnn_output_dim), merge_mode=bidir_mode, name='Bi-directional_LSTM_1'))
    model.add(Bidirectional(LSTM(rnn_hidden_dim, dropout=rnn_dropout), merge_mode=bidir_mode, name='Bi-directional_LSTM_2'))
    model.add(Dense(6, activation='softmax', name='Output_Dense'))
    
    return model

In [16]:
model = init_model3()

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Masking (TimeDistributed)    (None, 48, 21, 21, 1)     0         
_________________________________________________________________
Conv2D_1 (TimeDistributed)   (None, 48, 21, 21, 48)    816       
_________________________________________________________________
MaxPooling2D_1 (TimeDistribu (None, 48, 19, 19, 48)    0         
_________________________________________________________________
Conv2D_2 (TimeDistributed)   (None, 48, 19, 19, 24)    10392     
_________________________________________________________________
MaxPooling2D_2 (TimeDistribu (None, 48, 17, 17, 24)    0         
_________________________________________________________________
Conv2D_3 (TimeDistributed)   (None, 48, 17, 17, 12)    2604      
_________________________________________________________________
MaxPooling2D_3 (TimeDistribu (None, 48, 15, 15, 12)    0         
__________

In [14]:
opt = keras.optimizers.Adam(lr=5*1e-4)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
cpt = Callbacks.ModelCheckpoint(filepath='saved_models/rcnn_aug_ep100.hdf5', 
                      save_best_only=True, period=5, verbose=1)
tensorboard = Callbacks.TensorBoard(log_dir='./logs')

Instructions for updating:
Use the retry module or similar alternatives.


In [13]:
model = keras.models.load_model('saved_models/rcnn_aug_ep100.hdf5')

In [ ]:
history = model.fit(x, y, batch_size=100, epochs=60,
          validation_split=0.2, shuffle=False, callbacks=[cpt, tensorboard])

Train on 57600 samples, validate on 14400 samples
Epoch 1/60
57600/57600 [==============================] - 210s 4ms/step - loss: 0.4129 - acc: 0.8235 - val_loss: 0.4118 - val_acc: 0.8358
Epoch 2/60
57600/57600 [==============================] - 208s 4ms/step - loss: 0.4059 - acc: 0.8266 - val_loss: 0.3819 - val_acc: 0.8422
Epoch 3/60
57600/57600 [==============================] - 194s 3ms/step - loss: 0.4211 - acc: 0.8240 - val_loss: 0.3831 - val_acc: 0.8426
Epoch 4/60
 7000/57600 [==>...........................] - ETA: 2:48 - loss: 0.3936 - acc: 0.8376

In [ ]:
import json
with open('saved_history/rcnn_aug_history.json', 'w') as outfile:  
    json.dump(history.history, outfile)

In [183]:
tensorboard = Callbacks.TensorBoard(log_dir='./logs')

In [18]:
mod_rcnn_bidir = keras.models.load_model('saved_models/rcnn_ep200.hdf5')

In [19]:
print('Train: ', mod_rcnn_bidir.evaluate(x_train, y_train, batch_size=100))
print('Test: ', mod_rcnn_bidir.evaluate(x_test, y_test, batch_size=100))

57600/57600 [==============================] - 128s 2ms/step
Train:  [0.15347104696168876, 0.9448611137146751]
14400/14400 [==============================] - 31s 2ms/step
Test:  [0.15902564488351345, 0.9432638908425967]


In [ ]:
with open('saved_history/simple_lstm_ep300/history.json', 'r') as f:
    hist_ep100 = json.load(f)
with open('saved_history/simple_lstm_ep300/history1.json', 'r') as f:
    hist_ep200 = json.load(f)
with open('saved_history/simple_lstm_ep300/history2.json', 'r') as f:
    hist_ep300 = json.load(f)